O objetivo desse notebook é treinar um algoritmo em python que consiga fazer previsões com o máximo de exatidão possível sobre um dataset focado em sínteses(experimentais) realizadas por nossa equipe. Queremos máximizar a utilização do laboratório e de seus recursos.

Programadores: André Araújo (andre23035@ilum.cnpem.br) e Samira Oliveira (samira23016@ilum.cnpem.br)

Peço que caso outra pessoa utilize esse notebook referencie e entre em contato, obrigado!

In [2]:
# Bibliotecas

#Básicas
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

#Modelos
from sklearn.linear_model import LinearRegression 
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C, WhiteKernel

#Métricas
from sklearn.metrics import mean_absolute_error, mean_squared_error, root_mean_squared_error
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score


In [ ]:
# Funções

# Regressão Linear

def reglin(X, y, array = X, rmse = True):

    #array = array.reshape(1, -1)
    
    modelo = LinearRegression()
    modelo.fit(X, y)
    previsao = modelo.predict(array)

    if rmse == True:

        rmse = root_mean_squared_error(previsao, y)
        return previsao, rmse

    else:
        return previsao

def regpoli(X, y, array = X, rmse = True):

    modelo_linear = LinearRegression()
    modelo_polinomial = make_pipeline(PolynomialFeatures(5), modelo_linear)

    modelo_polinomial.fit(X, y)
    
    previsao = modelo_polinomial.predict(array)

    if rmse == True:

        rmse = root_mean_squared_error(previsao, y)
        return previsao, rmse

    else:
        return previsao

def gp(X, y, array = X, rmse = True):

    kernel = C(1.0, (1e-3, 1e6)) * RBF(length_scale=[1, 1, 1], length_scale_bounds=[(1e-4, 1e2), (1e-3, 1e3), (1e-3, 1e3)])# + WhiteKernel(noise_level=10e-5, noise_level_bounds=(1e-10, 1e-2))

    model = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=100, random_state=6574)

    model.fit(X, y)

    previsao, sigma = model.predict(array, return_std=True)
    
    if rmse == True:

        rmse = root_mean_squared_error(previsao, y)
        return previsao, rmse

    else:
        return previsao

In [8]:
# Tratamento de dados

df = pd.read_csv("dataset.csv", sep=';')

suas_colunas = ['[ureia]', 'ureia-p', '[y]', 'y-p', 'massa(g)','predicao','mse','media','desvio_p','polidisp']

# Iterando sobre as colunas e realizando a substituição
for coluna in suas_colunas:
    df[coluna] = df[coluna].str.replace(',', '.')

# Certifique-se de que os valores agora são tratados como números (floats)
df[suas_colunas] = df[suas_colunas].astype(float)

#Remoção das colunas não utilizadas ou sem importância.
df_alt1 = df.drop(["predicao", "mse", "Mol-Ureia",'Mol-Y2O3', "método", "balão(mL)"], axis=1)

#Remoção das linhas não utilizadas ou sem importância.
df_alt2 = df_alt1.drop([0, 1], axis=0)

#Reindexiando as linhas novamente
df_alt2 = df_alt2.reset_index(drop=True)

df_alt2

,samples,[ureia],ureia-p,[y],y-p,tempo(h),temp,massa(g),media,desvio_p,polidisp
0,SA-1,0.075,1.125188,0.02,2.2581,5,80,0.0300,488.03114,146.6063,0.300404
1,SA-2,0.075,1.125188,0.02,2.2581,5,85,0.1673,718.00950,145.4676,0.202598
2,SA-3,0.150,2.250375,0.02,2.2581,5,80,0.2316,455.41720,144.5251,0.317347
3,SA-4,0.150,2.250375,0.02,2.2581,5,85,0.4327,223.57090,26.1409,0.116924


In [9]:
# Definição de Features e Targets

FEATURES = ['[ureia]','tempo(h)', 'temp']
TARGET = ['media']

df1 = df_alt2.reindex(FEATURES + TARGET, axis=1)

X = df1.reindex(FEATURES, axis=1)
y = df1.reindex(TARGET, axis=1)

X = X.values
y = y.values.ravel()

In [12]:
# Separação de linhas que serão retiradas e treinadas.

df_test = df_alt2
df_conf = df_test
linhas = []
listas_rmse = []

previsoes_reglin = []
previsoes_regpoli = []
previsoes_gp = []

c = 0

for i in df_test['samples']:

    c = c

    linhas.append(c)

    c = c + 1


In [13]:
# Comitê

x = [[0.1,5,85]]

for l in linhas:
    
    print()
    print('-------------------Retirada e Treino--------------------')

    df_conf = df_test

    x_conf = 0

    x_conf = df_test.iloc[l]

    df_conf = df_test.drop([l], axis=0)
    
    # Definição de Features e Targets

    FEATURES = ['[ureia]','tempo(h)', 'temp']
    TARGET = ['media']

    x_conf = x_conf.reindex(FEATURES, axis=1)

    x_conf = x_conf.values

    x_real = x_conf.reshape(1, -1)

    df_conf_real = df_conf.reindex(FEATURES + TARGET, axis=1)

    X_conf = df_conf_real.reindex(FEATURES, axis=1)
    y_conf = df_conf_real.reindex(TARGET, axis=1)

    X_conf = X_conf.values
    y_conf = y_conf.values.ravel()


    print('linha retirada:', l)
    print('Previsão de reglin para linha',l,'é:',reglin(X_conf, y_conf, x, rmse=False))
    print('Previsão de regpoli para linha',l,'é:',regpoli(X_conf, y_conf, x, rmse=False))
    print('Previsão de gp para linha',l,'é:',gp(X_conf, y_conf, x, rmse=False))

    #previsões regressão linear
    previsoes_reglin.append(reglin(X_conf, y_conf, x, rmse=False))
    previsoes_regpoli.append(regpoli(X_conf, y_conf, x, rmse=False))
    previsoes_gp.append(gp(X_conf, y_conf, x, rmse=False))
    print()

    

    if l == max(linhas):
        print()
        print('-------------------Processo concluído--------------------')
        print()
        print(f'Para as features: {x}')
        print()
        print('A média das previsões de Regressão Linear é:', np.mean(previsoes_reglin))
        print('A média das previsões de Regressão Polinomial é:', np.mean(previsoes_regpoli))
        print('A média das previsões de Gaussian Process é:', np.mean(previsoes_gp))
        
        



-------------------Retirada e Treino--------------------
linha retirada: 0
Previsão de reglin para linha 0 é: [553.19663333]
Previsão de regpoli para linha 0 é: [553.19688996]
Previsão de gp para linha 0 é: [602.77361589]


-------------------Retirada e Treino--------------------
linha retirada: 1
Previsão de reglin para linha 1 é: [245.31352667]
Previsão de regpoli para linha 1 é: [251.2737058]
Previsão de gp para linha 1 é: [241.70691902]


-------------------Retirada e Treino--------------------
linha retirada: 2
Previsão de reglin para linha 2 é: [553.19663333]
Previsão de regpoli para linha 2 é: [553.19688996]
Previsão de gp para linha 2 é: [616.74142765]


-------------------Retirada e Treino--------------------
linha retirada: 3
Previsão de reglin para linha 3 é: [707.13818667]
Previsão de regpoli para linha 3 é: [704.15813151]
Previsão de gp para linha 3 é: [708.03133831]


-------------------Processo concluído--------------------

Para as features: [[0.1, 5, 85]]

A média das

In [ ]:
# Validação Cruzada

for l in linhas:

    df_conf = df_test

    x_conf = 0

    x_conf = df_test.iloc[l]

    df_conf = df_test.drop([l], axis=0)
    
    # Definição de Features e Targets

    FEATURES = ['[ureia]','tempo(h)', 'temp']
    TARGET = ['media']

    x_conf = x_conf.reindex(FEATURES, axis=1)

    x_conf = x_conf.values

    x_real = x_conf.reshape(1, -1)

    df_conf_real = df_conf.reindex(FEATURES + TARGET, axis=1)

    X_conf = df_conf_real.reindex(FEATURES, axis=1)
    y_conf = df_conf_real.reindex(TARGET, axis=1)

    X_conf = X_conf.values
    y_conf = y_conf.values.ravel()


    print('linha retirada:', l)
    print(X_conf)
    print('Previsão para linha',l,'é:',regpoli(X_conf, y_conf, x_real, rmse=False))
    previsoes.append(regpoli(X_conf, y_conf, x_real, rmse=False))

    if l == max(linhas):
        print('ok')
        df_conf = df_test
        


## Referências ##

1. Jupyterbook, 4 - Regressão Linear. Cassar, Daniel 2023.
2. Jupyterbook, 6.2 - Validação Cruzada. Cassar, Daniel 2023.
3. Jupyterbook, 10 - Detecção de valores anômalos. Cassar, Daniel 2023.
4. Blog, Comitê de classificadores, Classificação multiclasse e multirrótulo. Augusto, Felipe 2020.

## Agradecimentos ##

Primeiramente, gostariamos de agradecer o nosso excelentíssimo Professor da disciplina de Aprendizado de Máquina, Daniel Cassar, por ter nos guiado e dado as ferramentas necessárias para atingir nossos objetivos.

Além disso, a Ilum Escola de Ciência - CNPEM, por ter nos dado a oportunidade de desenvolver nossas habilidades acadêmicas com a oportunidade de fazer estágios e férias.

Por fim, a todos os conteúdos online que pudemos encontrar que nos ajudaram na nossa caminhada e no nosso desenvolvimento como programador.